In [1]:
import redis




redis_client = redis.Redis(
  host='redis-17025.c258.us-east-1-4.ec2.cloud.redislabs.com',
  port=17025,
  password='Nsq7jK8HD3Idhz2pFBtVd3xBusUmmOqt')

# redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)
# print(redis_client.ts().queryindex(['0xb3aca3081d10:battery']))

Redis Connected: True


In [2]:
import json
import paho.mqtt.client as mqtt
# Create a new MQTT client
client = mqtt.Client()
# Define the callbacks for when the client receives a response from the server
# and when a message is published on a subscribed topic
def on_connect(client, userdata, flags, rc):
 print(f'Connected with result code {str(rc)}')
 # Subscribe to a topic when the client connects
 client.subscribe('s305979')

created_timeseries = True

def on_message(client, userdata, msg):
    created_timeseries = True
    message = json.loads(msg.payload.decode())
    if(created_timeseries):
        created_timeseries = False 
        mac_battery = message['mac_address'] + ":battery" 
        mac_power =  message['mac_address'] + ":power"

        try:
            redis_client.ts().create(mac_battery)
        except redis.ResponseError:
        # Ignore error if the timeseries already exists
            pass
        try:
            redis_client.ts().create(mac_power)
        except redis.ResponseError:
        # Ignore error if the timeseries already exists
            pass
    redis_client.ts().add(mac_battery, message['timestamp'], message['battery_level'])
    redis_client.ts().add(mac_power, message['timestamp'], message['power_plugged'])
    print(f"Received message '{message}' on topic {msg.topic}")
# Set the callbacks




In [6]:
client.on_connect = on_connect
client.on_message = on_message
client.connect('mqtt.eclipseprojects.io', 1883)
# Start the client loop to process network events and call the callbacks
client.loop_forever()

Connected with result code 0


In [4]:
print(redis_client.ts().get('0xb3aca3081d10:battery'))
print(redis_client.ts().get('0xb3aca3081d10:power'))

(1673028887886, 98.0)
(1673028887886, 1.0)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1971cb16-e709-4b3f-8d6d-f6f260f3f6b7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>